# Order Pattern Analysis
## Info:
- params:
    - OPT_num: int, 할당해야 하는 OTP의 수
    - 별도설비_num: int, 할당해야 하는 별도설비 수
    - OTP_ratio: dict   
        = {A:[0.x, 0.x, 0.x, ...], B:[0.x, 0.x, 0.x, ..]}  
            - 비율 합이 100인지 여부는 추후 알려주시기로 함. -> 근데 웬만하면 100일 것 같아서 일단 100으로 작업..
    - SEP_ratio: list  
        = [0.x, 0.x, 0.x, ...]
    - Data: dataframe, raw data
    - Order: str, csv 파일 order 컬럼의 이름
    - Sku: str, csv 파일 sku code 컬럼의 이름
    - Rank: str, csv 파일 rank 컬럼의 이름

## Algorithm:
1. rank pattern 분류
    - pattern 1~7로 주문의 유형에 패턴 변수 부여하는 작업
    - 기존(2023)에 만들어진 함수를 재활용
2. equip analysis
    - manual하게 지정된 비율에 따라 균등하게...
    - 이후 라벨을 부여하기

In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("C:/Users/HARIN/Programming_PNU/2024_1_Capstone/data/OP_Sample_1203_raw_data.xlsx")

In [3]:
data

,ALLOCYN,ORDERDATE,ORDERKEY,UOM,ORDERKEY_SHORT,STORERKEY,SKU,SKUNAME,QTYEXPECTED,QTYPICKED,...,CONSIGNEEKEY,CUSTNAME,LOC,ExternOrderKey,DRIVERNAME,TYPE,PICKEDQTY,PACK_CNT,STATUS.1,Rank
0,NaN,2023-12-03,52912023120200005,1,00005,OL,8809864751781,컬러그램 쓱싹 오토 브로우 펜,1,1,...,22,최은비,A07-1704,Y2312010884081,582477261264,X,0,1,9,C
1,NaN,2023-12-03,52912023120200005,1,00005,OL,8809838651697,닥터지 브라이트닝 필링젤 120g,1,1,...,22,최은비,A19-0201,Y2312010884081,582477261264,L,0,1,9,A
2,NaN,2023-12-03,52912023120200005,1,00005,OL,8809409340104,메디필 엑스트라 슈퍼9 플러스,1,1,...,22,최은비,A14-6604,Y2312010884081,582477261264,L,0,1,9,A
3,NaN,2023-12-03,52912023120200005,1,00005,OL,8809937598336,클리오 킬래쉬 수퍼프루프마,1,1,...,22,최은비,A12-4501,Y2312010884081,582477261264,L,0,1,9,B
4,NaN,2023-12-03,52912023120200008,1,00008,OL,8809598456129,코스알엑스 더 비타민C 23 세럼,1,1,...,12,김민선,A05-5704,Y2312010880932,582477261290,L,0,1,9,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123385,NaN,2023-12-03,529120231130D9145,1,D9145,OL,8809900986030,페리페라 워터베어틴트 3.7g 008,1,1,...,22,윤채미,V15-0404,Y2311300555316,582475873190,L,0,1,9,B
123386,NaN,2023-12-03,529120231130D9145,1,D9145,OL,8809238964823,아로마티카 퓨어 앤 소프트 여,1,1,...,22,윤채미,A03-2001,Y2311300555316,582475873190,L,0,1,9,B
123387,NaN,2023-12-03,529120231130D9145,1,D9145,OL,3282770208702,클로란 네틀 노세범 드라이 샴,1,1,...,22,윤채미,D19-3005,Y2311300555316,582475873190,L,0,1,9,B
123388,NaN,2023-12-03,529120231130D9145,1,D9145,OL,8801166222600,좋은느낌라이너 유기농순면,1,1,...,22,윤채미,A03-1902,Y2311300555316,582475873190,L,0,1,9,B


In [4]:
data.columns

Index(['ALLOCYN', 'ORDERDATE', 'ORDERKEY', 'UOM', 'ORDERKEY_SHORT',
       'STORERKEY', 'SKU', 'SKUNAME', 'QTYEXPECTED', 'QTYPICKED', 'STATUS',
       'ADDDATE', 'EDITDATE', 'WORK_YN', 'CONSIGNEEKEY', 'CUSTNAME', 'LOC',
       'ExternOrderKey', 'DRIVERNAME', 'TYPE', 'PICKEDQTY', 'PACK_CNT',
       'STATUS.1', 'Rank'],
      dtype='object')

In [5]:
# params
order = 'ORDERKEY'
sku = 'SKU'
rank = 'Rank'
quantity = 'QTYEXPECTED'
OTP_ratio = {
    'A': [0.2, 0.3, 0.5],
    'B': [0.3, 0.35, 0.35]
}
SEP_ratio = [0.3, 0.7]

In [6]:
new_data = data[['ORDERKEY', 'SKU', 'QTYEXPECTED', 'Rank']]
new_data

,ORDERKEY,SKU,QTYEXPECTED,Rank
0,52912023120200005,8809864751781,1,C
1,52912023120200005,8809838651697,1,A
2,52912023120200005,8809409340104,1,A
3,52912023120200005,8809937598336,1,B
4,52912023120200008,8809598456129,1,A
...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B
123386,529120231130D9145,8809238964823,1,B
123387,529120231130D9145,3282770208702,1,B
123388,529120231130D9145,8801166222600,1,B


## 1. rank pattern 분류: 2023 ver.

In [7]:
def rank_analysis(data, rank, order):
        import pandas as pd
        import numpy as np
        import time, datetime
        start = time.time()

        ################# 주문패턴 (Rank pattern) 부여 #################
        op_frame = data.groupby(order)[rank].unique().to_frame()
        op_frame['Rank패턴']=np.nan
        for i in range(len(op_frame)):
            rank_list = op_frame.iloc[i][rank]
    
            if 'A' in rank_list:
                if 'B' in rank_list:
                    if 'C' in rank_list:
                        op_frame.iloc[i,1]='pattern7'
                    else:
                        op_frame.iloc[i,1]='pattern4'
                elif 'C' in rank_list:
                    op_frame.iloc[i,1]='pattern5'
                else:
                    op_frame.iloc[i,1]='pattern1'
            elif 'B' in rank_list:
                if 'C' in rank_list:
                    op_frame.iloc[i,1]='pattern6'
                else:
                    op_frame.iloc[i,1]='pattern2'
            else:
                op_frame.iloc[i,1]='pattern3'

        try:
            data['Rank패턴'] = data[order]
            for i in range(len(data)):
                data.iloc[i,-1] = op_frame.loc[data.iloc[i,-1]][1]  
            print('Rank 패턴 분류를 완료하였습니다.')
            
            end = time.time()
            print('소요된 시간: ',datetime.timedelta(seconds=end-start))
            return data
        except:
            print('Rank 패턴 분류에 문제가 생겼습니다. 데이터를 다시 확인해주세요.')
            return

In [8]:
ranked  = rank_analysis(new_data, rank, order)

C:\Users\HARIN\AppData\Local\Temp\ipykernel_16304\701031293.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rank패턴'] = data[order]


Rank 패턴 분류를 완료하였습니다.
소요된 시간:  0:00:26.768237


In [9]:
ranked

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴
0,52912023120200005,8809864751781,1,C,pattern7
1,52912023120200005,8809838651697,1,A,pattern7
2,52912023120200005,8809409340104,1,A,pattern7
3,52912023120200005,8809937598336,1,B,pattern7
4,52912023120200008,8809598456129,1,A,pattern5
...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7
123386,529120231130D9145,8809238964823,1,B,pattern7
123387,529120231130D9145,3282770208702,1,B,pattern7
123388,529120231130D9145,8801166222600,1,B,pattern7


In [10]:
ranked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123390 entries, 0 to 123389
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ORDERKEY     123390 non-null  object
 1   SKU          123390 non-null  int64 
 2   QTYEXPECTED  123390 non-null  int64 
 3   Rank         123390 non-null  object
 4   Rank패턴       123390 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.7+ MB


In [11]:
# ranked = ranked.astype({'ORDERKEY':'string', 'Rank':'string', 'Rank패턴':'string'})

In [12]:
ranked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123390 entries, 0 to 123389
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ORDERKEY     123390 non-null  object
 1   SKU          123390 non-null  int64 
 2   QTYEXPECTED  123390 non-null  int64 
 3   Rank         123390 non-null  object
 4   Rank패턴       123390 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.7+ MB


## 2. equip analysis

4. equip analysis for A, B
    1. 품목으로 groupby -> 비율 맞게 할당
    2. 작업량 기준 정해두고 안맞다면 다른 OTP와 변경
        - 총 작업량: 행수*수량(A, B 같이)
        - 결국은 이게 잘 안맞을 수 있는 거니까... 
    3. 이게 잘 맞는다는 어떤 범위를 충족한다면 픽스
    4. C에 대한 것도 수행
    5. 처음의 데이터셋에 주문번호별로 sku를 기준으로 해서 어떤 설비에 들어갈지 표시하는 열 만들기
    6. 그걸 기준으로 주문번호별로 A, B, C 각 랭크 물건에 대하여 설비의 패턴 할당. 끝.

In [13]:
OTP = ranked[ranked.Rank!='C']
OTP

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴
1,52912023120200005,8809838651697,1,A,pattern7
2,52912023120200005,8809409340104,1,A,pattern7
3,52912023120200005,8809937598336,1,B,pattern7
4,52912023120200008,8809598456129,1,A,pattern5
5,52912023120200008,8809693601004,1,A,pattern5
...,...,...,...,...,...
123384,529120231130D9145,8809963500471,1,A,pattern7
123385,529120231130D9145,8809900986030,1,B,pattern7
123386,529120231130D9145,8809238964823,1,B,pattern7
123387,529120231130D9145,3282770208702,1,B,pattern7


In [14]:
OTP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98703 entries, 1 to 123388
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ORDERKEY     98703 non-null  object
 1   SKU          98703 non-null  int64 
 2   QTYEXPECTED  98703 non-null  int64 
 3   Rank         98703 non-null  object
 4   Rank패턴       98703 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [15]:
SEP = ranked[ranked.Rank=='C']
SEP

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴
0,52912023120200005,8809864751781,1,C,pattern7
7,52912023120200008,8809438619981,1,C,pattern5
9,52912023120200012,8809495894000,9,C,pattern6
10,52912023120200012,8806390524612,2,C,pattern6
20,52912023120100020,8809517473244,1,C,pattern7
...,...,...,...,...,...
123354,529120231130D7374,8850956162374,1,C,pattern7
123361,529120231130D7943,8809925124233,1,C,pattern7
123365,529120231130D8209,8809990335695,1,C,pattern7
123375,529120231130D8393,8809971480345,1,C,pattern7


In [16]:
OTP_qty = int(OTP[quantity].sum()/len(OTP_ratio['A']))
OTP_qty

37916

In [17]:
SEP_qty = int(SEP[quantity].sum()/len(SEP_ratio))
SEP_qty

14623

In [18]:
OTP_A = OTP[OTP['Rank']=='A']
OTP_B = OTP[OTP['Rank']=='B']

In [19]:
OTP_A_grouped = pd.DataFrame(OTP_A.groupby(sku)[quantity].sum()).reset_index()
OTP_B_grouped = pd.DataFrame(OTP_B.groupby(sku)[quantity].sum()).reset_index()
SEP_grouped = pd.DataFrame(SEP.groupby(sku)[quantity].sum()).reset_index()

In [20]:
OTP_A_grouped

,SKU,QTYEXPECTED
0,80321170,89
1,41388001715,127
2,192333250549,168
3,192333250563,64
4,669416496502,72
...,...,...
437,8901138509231,168
438,9300807058275,106
439,9343202008810,77
440,9555074314993,48


In [21]:
OTP_B_grouped

,SKU,QTYEXPECTED
0,30106659,16
1,80770442,45
2,94204513,12
3,10700021526,38
4,18787506943,18
...,...,...
2320,9555074314757,22
2321,9555074314764,12
2322,9555074314900,25
2323,9555074314917,19


In [22]:
SEP_grouped

,SKU,QTYEXPECTED
0,40144016,18
1,42332619,5
2,80796565,2
3,80820994,10
4,80845386,7
...,...,...
5091,9555074314733,3
5092,9555074314740,6
5093,9555074314887,2
5094,9555074314894,10


In [23]:
OTP_ratio = {
    'A': [1/3, 1/3, 1/3],
    'B': [0.2, 0.7, 0.1]
}
SEP_ratio = [1]

In [24]:
def initial_split_sku(grouped, ratios):
    import numpy as np
    shuffled = np.random.permutation(grouped.index) # 랜덤성 부여
    row_num = len(grouped)
    group_sizes = [int(row_num * ratio) for ratio in ratios]
    splits = np.split(shuffled, np.cumsum(group_sizes)[:-1]) # 해당 랭크의 비율에 맞게 쪼개기
    splited = [grouped.iloc[split] for split in splits]
    splited_sorted = [splited_frag.sort_values(by='QTYEXPECTED', ascending=False) for splited_frag in splited]
    return splited_sorted

In [25]:
splited_A = initial_split_sku(OTP_A_grouped, OTP_ratio['A'])
splited_B = initial_split_sku(OTP_B_grouped, OTP_ratio['B'])
splited_C = initial_split_sku(SEP_grouped, SEP_ratio)

In [26]:
for i in range(len(splited_A)):
    print(splited_A[i]['QTYEXPECTED'].sum())

19686
17676
18704


In [27]:
for i in range(len(splited_B)):
    print(splited_B[i]['QTYEXPECTED'].sum())

11137
40916
5629


In [28]:
for i in range(len(splited_C)):
    print(splited_C[i]['QTYEXPECTED'].sum())

29246


In [29]:
for i in range(len(splited_B)):
    print(splited_A[i]['QTYEXPECTED'].sum()+splited_B[i]['QTYEXPECTED'].sum())

30823
58592
24333


In [30]:
def quantity_adjust_OTP(splited_A, splited_B, tolerance=0.05, max_iter=10**5):
    import pandas as pd
    import numpy as np
    import time, datetime
    start = time.time()
    
    # qty 계산
    A_qty_list = [i[quantity].sum() for i in splited_A]
    B_qty_list = [i[quantity].sum() for i in splited_B]
    total_qty_list = [A_qty_list[i]+B_qty_list[i] for i in range(len(splited_A))]
    ideal_qty = int(sum(total_qty_list)/len(total_qty_list))
    A_sku_min = min([len(i[sku]) for i in splited_A])
    B_sku_min = min([len(i[sku]) for i in splited_B])
    
    iteration = 0
    
    while iteration<=max_iter: # test 데이터 기준으로 다 도는 데 약 4분 소요.
        # max와 min 그룹의 인덱스 구하기
        min_idx = total_qty_list.index(min(total_qty_list))
        max_idx = total_qty_list.index(max(total_qty_list))
        
        # 차이 계산
        diff_min = abs(ideal_qty-total_qty_list[min_idx])/ideal_qty
        diff_max = abs(ideal_qty-total_qty_list[max_idx])/ideal_qty

        # qty값 조정
        if diff_min>tolerance or diff_max>tolerance:
            # 교환 비율 지정: 10%로!
            slice_min_A = int(A_sku_min*max(diff_min, diff_max))
            slice_min_B = int(B_sku_min*max(diff_min, diff_max))
            
            slice_min_A = int(A_sku_min*0.1)
            slice_min_B = int(B_sku_min*0.1)
            # 교환
            splited_A[min_idx].iloc[-(slice_min_A+1):-1], splited_A[max_idx].iloc[0:slice_min_A] = splited_A[max_idx].iloc[0:slice_min_A].copy(), splited_A[min_idx].iloc[-(slice_min_A+1):-1].copy()
            splited_B[min_idx].iloc[-(slice_min_B+1):-1], splited_B[max_idx].iloc[0:slice_min_B] = splited_B[max_idx].iloc[0:slice_min_B].copy(), splited_B[min_idx].iloc[-(slice_min_B+1):-1].copy()
        else:
            print(f"Alert: Ideally divided! (tolerance={tolerance*100}%)\n")
            break
        
        # 다음 반복 위한 처리
        A_qty_list = [i[quantity].sum() for i in splited_A]
        B_qty_list = [i[quantity].sum() for i in splited_B]
        total_qty_list = [A_qty_list[i]+B_qty_list[i] for i in range(len(splited_A))]        
        splited_A = [frag.sort_values(by='QTYEXPECTED', ascending=False) for frag in splited_A]
        splited_B = [frag.sort_values(by='QTYEXPECTED', ascending=False) for frag in splited_B]
        
        iteration += 1
        
    end = time.time()
    
    ##### results
    if iteration>max_iter: print(f"Alert: May not be equally divided (tolerance={tolerance*100}%)\nminimum difference: {diff_min}, maximum difference: {diff_max}\n")
    print('QUANTITY')
    print('====================')
    print(f'IDEAL: {ideal_qty}')
    for i in range(len(total_qty_list)):
        qty = splited_A[i]['QTYEXPECTED'].sum()+splited_B[i]['QTYEXPECTED'].sum()
        print(f'OTP {i+1}: {qty}')
    print('====================')
    
    print('')
    print('TIME')
    print('====================')
    print(datetime.timedelta(seconds=end-start))
    print('====================')
    
    return splited_A, splited_B

In [31]:
final_A, final_B = quantity_adjust_OTP(splited_A, splited_B)

Alert: May not be equally divided (tolerance=5.0%)
minimum difference: 0.04422934908745648, maximum difference: 0.07540352357843655

QUANTITY
IDEAL: 37916
OTP 1: 36734
OTP 2: 40604
OTP 3: 36410

TIME
0:06:03.897444


In [32]:
def quantity_adjust_SEP(splited_C, tolerance=0.05, max_iter=10**5):
    import pandas as pd
    import numpy as np
    import time, datetime
    start = time.time()
    
    # qty 계산
    qty_list = [i[quantity].sum() for i in splited_C]
    ideal_qty = int(sum(qty_list)/len(qty_list))
    sku_min = min([len(i[sku]) for i in splited_C])
    
    iteration = 0
    
    while iteration<=max_iter: # test 데이터 기준으로 다 도는 데 약 4분 소요.
        # max와 min 그룹의 인덱스 구하기
        min_idx = qty_list.index(min(qty_list))
        max_idx = qty_list.index(max(qty_list))
        
        # 차이 계산
        diff_min = abs(ideal_qty-qty_list[min_idx])/ideal_qty
        diff_max = abs(ideal_qty-qty_list[max_idx])/ideal_qty

        # qty값 조정
        if diff_min>tolerance or diff_max>tolerance:
            # 교환 비율 지정: 10%로!            
            slice_min = int(sku_min*0.1)
            # 교환
            splited_C[min_idx].iloc[-(slice_min+1):-1], splited_C[max_idx].iloc[0:slice_min] = splited_C[max_idx].iloc[0:slice_min].copy(), splited_C[min_idx].iloc[-(slice_min+1):-1].copy()
        else:
            print(f"Alert: Ideally divided! (tolerance={tolerance*100}%)\n")
            break
        
        # 다음 반복 위한 처리
        qty_list = [i[quantity].sum() for i in splited_C]
        splited_C = [frag.sort_values(by='QTYEXPECTED', ascending=False) for frag in splited_C]
        
        iteration += 1
        
    end = time.time()
    
    ##### results
    if iteration>max_iter: print(f"Alert: May not be equally divided (tolerance={tolerance*100}%)\nminimum difference: {diff_min}, maximum difference: {diff_max}\n")
    print('QUANTITY')
    print('====================')
    print(f'IDEAL: {ideal_qty}')
    for i in range(len(qty_list)):
        qty = splited_C[i]['QTYEXPECTED'].sum()
        print(f'SEP {i+1}: {qty}')
    print('====================')
    
    print('')
    print('TIME')
    print('====================')
    print(datetime.timedelta(seconds=end-start))
    print('====================')
    
    return splited_C

In [33]:
final_C = quantity_adjust_SEP(splited_C)

Alert: Ideally divided! (tolerance=5.0%)

QUANTITY
IDEAL: 29246
SEP 1: 29246

TIME
0:00:00.022665


### equip analysis - 패턴 라벨링

In [34]:
equips='OTP'
ranks='A'

In [106]:
grouped_A = []
for num, df in enumerate(final_A):
    final_A[num][f'{equips}_{ranks}'] = f'{equips} {num+1}'
    grouped_A.append(final_A[num])
    
grouped_A = pd.concat(grouped_A, ignore_index=True)[['SKU', f'{equips}_{ranks}']]

In [107]:
grouped_A

,SKU,OTP_A
0,8809241341550,OTP 1
1,8809324922478,OTP 1
2,8809669502915,OTP 1
3,8809971480178,OTP 1
4,8809510000836,OTP 1
...,...,...
437,8809213808883,OTP 3
438,8809541377044,OTP 3
439,8809664986802,OTP 3
440,8809897420234,OTP 3


In [108]:
merged_A = ranked.merge(grouped_A, on='SKU', how='left')
merged_A

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,OTP_A
0,52912023120200005,8809864751781,1,C,pattern7,NaN
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3
3,52912023120200005,8809937598336,1,B,pattern7,NaN
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1
...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,NaN
123386,529120231130D9145,8809238964823,1,B,pattern7,NaN
123387,529120231130D9145,3282770208702,1,B,pattern7,NaN
123388,529120231130D9145,8801166222600,1,B,pattern7,NaN


In [109]:
grouped_B = []
for num, df in enumerate(final_B):
    final_B[num]['OTP_B'] = f'OTP {num+1}'
    grouped_B.append(final_B[num])
    
grouped_B = pd.concat(grouped_B, ignore_index=True)[['SKU', 'OTP_B']]
grouped_B

,SKU,OTP_B
0,8809348502793,OTP 1
1,8809598453258,OTP 1
2,8809784601241,OTP 1
3,8996001330470,OTP 1
4,8809027558738,OTP 1
...,...,...
2320,8801201063021,OTP 3
2321,8801013350951,OTP 3
2322,8809568932493,OTP 3
2323,8809367311451,OTP 3


In [110]:
merged_B = merged_A.merge(grouped_B, on='SKU', how='left')
merged_B

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,OTP_A,OTP_B
0,52912023120200005,8809864751781,1,C,pattern7,NaN,NaN
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1,NaN
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3,NaN
3,52912023120200005,8809937598336,1,B,pattern7,NaN,OTP 1
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1,NaN
...,...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,NaN,OTP 2
123386,529120231130D9145,8809238964823,1,B,pattern7,NaN,OTP 2
123387,529120231130D9145,3282770208702,1,B,pattern7,NaN,OTP 3
123388,529120231130D9145,8801166222600,1,B,pattern7,NaN,OTP 1


In [111]:
grouped_C = []
for num, df in enumerate(final_C):
    final_C[num]['SEP_C'] = f'SEP {num+1}'
    grouped_C.append(final_C[num])
    
grouped_C = pd.concat(grouped_C, ignore_index=True)[['SKU', 'SEP_C']]
grouped_C

,SKU,SEP_C
0,8801051108880,SEP 1
1,8809382883841,SEP 1
2,8801092519713,SEP 1
3,8809520468534,SEP 1
4,8809615057087,SEP 1
...,...,...
5091,8809350089589,SEP 1
5092,8801051459586,SEP 1
5093,8809783140826,SEP 1
5094,4901433038539,SEP 1


In [112]:
merged_C = merged_B.merge(grouped_C, on='SKU', how='left')
merged_C

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,OTP_A,OTP_B,SEP_C
0,52912023120200005,8809864751781,1,C,pattern7,NaN,NaN,SEP 1
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1,NaN,NaN
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3,NaN,NaN
3,52912023120200005,8809937598336,1,B,pattern7,NaN,OTP 1,NaN
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1,NaN,NaN
...,...,...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,NaN,OTP 2,NaN
123386,529120231130D9145,8809238964823,1,B,pattern7,NaN,OTP 2,NaN
123387,529120231130D9145,3282770208702,1,B,pattern7,NaN,OTP 3,NaN
123388,529120231130D9145,8801166222600,1,B,pattern7,NaN,OTP 1,NaN


In [113]:
merged_C['EQUIP'] = merged_C['OTP_A'].combine_first(merged_C['OTP_B']).combine_first(merged_C['SEP_C'])
merged_C

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,OTP_A,OTP_B,SEP_C,EQUIP
0,52912023120200005,8809864751781,1,C,pattern7,NaN,NaN,SEP 1,SEP 1
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1,NaN,NaN,OTP 1
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3,NaN,NaN,OTP 3
3,52912023120200005,8809937598336,1,B,pattern7,NaN,OTP 1,NaN,OTP 1
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1,NaN,NaN,OTP 1
...,...,...,...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,NaN,OTP 2,NaN,OTP 2
123386,529120231130D9145,8809238964823,1,B,pattern7,NaN,OTP 2,NaN,OTP 2
123387,529120231130D9145,3282770208702,1,B,pattern7,NaN,OTP 3,NaN,OTP 3
123388,529120231130D9145,8801166222600,1,B,pattern7,NaN,OTP 1,NaN,OTP 1


In [114]:
order_categories = merged_C.groupby('ORDERKEY')['EQUIP'].apply(lambda x: ''.join(sorted(set(x)))).reset_index()
order_categories['EQUIP 패턴'] = 'pattern' + (order_categories.groupby('EQUIP').ngroup()+1).astype(str)

final_DF = merged_C.merge(order_categories[['ORDERKEY', 'EQUIP 패턴']], on='ORDERKEY')

In [125]:
final_DF

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,OTP_A,OTP_B,SEP_C,EQUIP,EQUIP 패턴
0,52912023120200005,8809864751781,1,C,pattern7,NaN,NaN,SEP 1,SEP 1,pattern7
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1,NaN,NaN,OTP 1,pattern7
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3,NaN,NaN,OTP 3,pattern7
3,52912023120200005,8809937598336,1,B,pattern7,NaN,OTP 1,NaN,OTP 1,pattern7
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1,NaN,NaN,OTP 1,pattern7
...,...,...,...,...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,NaN,OTP 2,NaN,OTP 2,pattern4
123386,529120231130D9145,8809238964823,1,B,pattern7,NaN,OTP 2,NaN,OTP 2,pattern4
123387,529120231130D9145,3282770208702,1,B,pattern7,NaN,OTP 3,NaN,OTP 3,pattern4
123388,529120231130D9145,8801166222600,1,B,pattern7,NaN,OTP 1,NaN,OTP 1,pattern4


In [126]:
final_DF.columns

Index(['ORDERKEY', 'SKU', 'QTYEXPECTED', 'Rank', 'Rank패턴', 'OTP_A', 'OTP_B',
       'SEP_C', 'EQUIP', 'EQUIP 패턴'],
      dtype='object')

In [127]:
save_name = 'OP_result'

In [128]:
use = ['ORDERKEY', 'SKU', 'QTYEXPECTED', 'Rank', 'Rank패턴', 'EQUIP', 'EQUIP 패턴']

In [129]:
final = final_DF[use]
print(final.info())
final = final.astype({'SKU':'string'})
print(final.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 123390 entries, 0 to 123389
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ORDERKEY     123390 non-null  object
 1   SKU          123390 non-null  int64 
 2   QTYEXPECTED  123390 non-null  int64 
 3   Rank         123390 non-null  object
 4   Rank패턴       123390 non-null  object
 5   EQUIP        123390 non-null  object
 6   EQUIP 패턴     123390 non-null  object
dtypes: int64(2), object(5)
memory usage: 7.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 123390 entries, 0 to 123389
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ORDERKEY     123390 non-null  object
 1   SKU          123390 non-null  string
 2   QTYEXPECTED  123390 non-null  int64 
 3   Rank         123390 non-null  object
 4   Rank패턴       123390 non-null  object
 5   EQUIP        123390 non-null  object
 6   EQUIP 

In [130]:
final.to_excel(f'{save_name}.xlsx', index=False, encoding='cp949', engine='openpyxl')

C:\Users\HARIN\Anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [135]:
final

,ORDERKEY,SKU,QTYEXPECTED,Rank,Rank패턴,EQUIP,EQUIP 패턴
0,52912023120200005,8809864751781,1,C,pattern7,SEP 1,pattern7
1,52912023120200005,8809838651697,1,A,pattern7,OTP 1,pattern7
2,52912023120200005,8809409340104,1,A,pattern7,OTP 3,pattern7
3,52912023120200005,8809937598336,1,B,pattern7,OTP 1,pattern7
4,52912023120200008,8809598456129,1,A,pattern5,OTP 1,pattern7
...,...,...,...,...,...,...,...
123385,529120231130D9145,8809900986030,1,B,pattern7,OTP 2,pattern4
123386,529120231130D9145,8809238964823,1,B,pattern7,OTP 2,pattern4
123387,529120231130D9145,3282770208702,1,B,pattern7,OTP 3,pattern4
123388,529120231130D9145,8801166222600,1,B,pattern7,OTP 1,pattern4


In [134]:
order_categories[order_categories['EQUIP 패턴']=='pattern5']

,ORDERKEY,EQUIP,EQUIP 패턴
2,52912023113017387,OTP 1OTP 2SEP 1,pattern5
3,52912023113018382,OTP 1OTP 2SEP 1,pattern5
9,52912023113028862,OTP 1OTP 2SEP 1,pattern5
11,52912023113032548,OTP 1OTP 2SEP 1,pattern5
20,52912023113048558,OTP 1OTP 2SEP 1,pattern5
...,...,...,...
25543,529120231202B4419,OTP 1OTP 2SEP 1,pattern5
25546,529120231202B4478,OTP 1OTP 2SEP 1,pattern5
25550,529120231202B4552,OTP 1OTP 2SEP 1,pattern5
25551,529120231202B4557,OTP 1OTP 2SEP 1,pattern5


In [132]:
order_categories['EQUIP 패턴'].unique()

array(['pattern4', 'pattern5', 'pattern12', 'pattern3', 'pattern11',
       'pattern7', 'pattern2', 'pattern10', 'pattern9', 'pattern14',
       'pattern8', 'pattern6', 'pattern15', 'pattern13', 'pattern1'],
      dtype=object)